In [5]:
import pandas as pd
import numpy as np
import sklearn as sk
import sklearn.neural_network as sknn

In [6]:
datos = pd.read_csv("./kaggle-4-datasets-12-atrib.csv")
datos

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [7]:
datos['Age'] = (datos['Age'] - datos['Age'].min()) / (datos['Age'].max() - datos['Age'].min())
datos['Sex'] = datos['Sex'].replace({'F':1.0, 'M':0.0})
datos['ChestPainType'] = datos['ChestPainType'].replace({'ASY':0.0, 'ATA':0.333, 'NAP': 0.667, 'TA': 1.0}) 
datos['RestingBP'] = (datos['RestingBP'] - datos['RestingBP'].min()) / (datos['RestingBP'].max() - datos['RestingBP'].min())
datos['Cholesterol'] = (datos['Cholesterol'] - datos['Cholesterol'].min()) / (datos['Cholesterol'].max() - datos['Cholesterol'].min())
datos['RestingECG'] = datos['RestingECG'].replace({'LVH':0.0, 'Normal':0.5, 'ST': 1.0}) 
datos['MaxHR'] = (datos['MaxHR'] - datos['MaxHR'].min()) / (datos['MaxHR'].max() - datos['MaxHR'].min())
datos['ExerciseAngina'] = datos['ExerciseAngina'].replace({'N':0.0, 'Y':1.0})
datos['Oldpeak'] = (datos['Oldpeak'] - datos['Oldpeak'].min()) / (datos['Oldpeak'].max() - datos['Oldpeak'].min())
datos['ST_Slope'] = datos['ST_Slope'].replace({'Down':0.0, 'Flat':0.5, 'Up': 1.0})

outputs = np.array(datos['HeartDisease'])
inputs = np.array(datos.drop(['HeartDisease'], axis=1))


inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

<IPython.core.display.Javascript object>

In [8]:
def run_nn(inputs_train, inputs_test, outputs_train, outputs_test, h_layers, verbose=False):
    # Create the model
    model = sknn.MLPClassifier(
        hidden_layer_sizes=h_layers, 
        max_iter=5000, 
        alpha=0.0001, 
        solver='lbfgs', 
        learning_rate='adaptive',
        n_iter_no_change=500,
        verbose=verbose, 
        tol=0.000000001, 
        random_state=1
    )
    # model = sknn.MLPRegressor(solver='sgd', batch_size='auto', tol=0.0005, learning_rate_init=0.002, hidden_layer_sizes=h_layers, verbose=verbose)
    # Train the model
    model.fit(inputs_train, outputs_train)
    # Predict the test set
    
    acc = 0
    true_true = 0
    true_false = 0
    false_true = 0
    false_false = 0
    
    for i in range(0, len(inputs_test)):
        pre = round(model.predict(inputs_test[i].reshape(1, -1))[0], 0)
        true_true = true_true + (pre == outputs_test[i] and outputs_test[i] == 1)
        true_false = true_false + (pre == outputs_test[i] and outputs_test[i] == 0)
        false_true = false_true + (pre != outputs_test[i] and outputs_test[i] == 1)
        false_false = false_false + (pre != outputs_test[i] and outputs_test[i] == 0)
        acc = acc + (pre == outputs_test[i])
        success = "#" if pre == outputs_test[i] else "."
        if verbose:
            print(success, end="")

    acc = acc / len(inputs_test)
    precision = true_true / (true_true + false_true)
    recall = true_true / (true_true + false_false)
    f1 = 2 * (precision * recall) / (precision + recall)
    # print("\nAccuracy:", acc)
    return {'acc': acc, 'precision': precision, 'recall': recall, 'f1': f1}

In [ ]:
import tensorflow as tf

In [9]:
import itertools

max_nhl =  4    # max number of hidden layers
neurons = [10, 20, 40, 80, 160]
res = {}
m = {'acc': 0, 'precision': 0, 'recall': 0, 'f1': 0}

def max(a, b):
    a['acc'] = b['acc'] if b['acc'] > a['acc'] else a['acc']
    a['precision'] = b['precision'] if b['precision'] > a['precision'] else a['precision']
    a['recall'] = b['recall'] if b['recall'] > a['recall'] else a['recall']
    a['f1'] = b['f1'] if b['f1'] > a['f1'] else a['f1'] 

for nhl in range(1, max_nhl+1):
    for subset in itertools.product(neurons, repeat=nhl):
        print('Procsessing:', subset, end=" ")
        metricas = run_nn(inputs_train, inputs_test, outputs_train, outputs_test, subset, verbose=False)
        max(m, metricas)
        res[subset] = metricas
        print('Acc: %.3f, Precision: %.3f, Recall: %.3f, F1: %.3f' % (metricas['acc'], metricas['precision'], metricas['recall'], metricas['f1']), end=" ")
        print('Max: Acc: %.3f' % (m['acc']))


print('res:', res)
print('m:', m)

Procsessing: (10,) Acc: 0.891, Precision: 0.907, Recall: 0.907, F1: 0.907 Max: Acc: 0.891
Procsessing: (20,) Acc: 0.772, Precision: 0.757, Recall: 0.835, F1: 0.794 Max: Acc: 0.891
Procsessing: (40,) Acc: 0.821, Precision: 0.748, Recall: 0.930, F1: 0.829 Max: Acc: 0.891
Procsessing: (80,) Acc: 0.837, Precision: 0.804, Recall: 0.905, F1: 0.851 Max: Acc: 0.891
Procsessing: (160,) Acc: 0.799, Precision: 0.785, Recall: 0.857, F1: 0.820 Max: Acc: 0.891
Procsessing: (10, 10) Acc: 0.804, Precision: 0.794, Recall: 0.859, F1: 0.825 Max: Acc: 0.891
Procsessing: (10, 20) Acc: 0.799, Precision: 0.794, Recall: 0.850, F1: 0.821 Max: Acc: 0.891
Procsessing: (10, 40) Acc: 0.739, Precision: 0.692, Recall: 0.831, F1: 0.755 Max: Acc: 0.891
Procsessing: (10, 80) Acc: 0.799, Precision: 0.757, Recall: 0.880, F1: 0.814 Max: Acc: 0.891
Procsessing: (10, 160) Acc: 0.842, Precision: 0.804, Recall: 0.915, F1: 0.856 Max: Acc: 0.891
Procsessing: (20, 10) Acc: 0.783, Precision: 0.748, Recall: 0.860, F1: 0.800 Max: A

/home/rolando/anaconda3/envs/healt/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Acc: 0.799, Precision: 0.748, Recall: 0.889, F1: 0.812 Max: Acc: 0.891
Procsessing: (10, 20, 160) Acc: 0.799, Precision: 0.776, Recall: 0.865, F1: 0.818 Max: Acc: 0.891
Procsessing: (10, 40, 10) Acc: 0.821, Precision: 0.794, Recall: 0.885, F1: 0.837 Max: Acc: 0.891
Procsessing: (10, 40, 20) Acc: 0.815, Precision: 0.813, Recall: 0.861, F1: 0.837 Max: Acc: 0.891
Procsessing: (10, 40, 40) Acc: 0.761, Precision: 0.692, Recall: 0.871, F1: 0.771 Max: Acc: 0.891
Procsessing: (10, 40, 80) Acc: 0.783, Precision: 0.738, Recall: 0.868, F1: 0.798 Max: Acc: 0.891
Procsessing: (10, 40, 160) Acc: 0.826, Precision: 0.813, Recall: 0.879, F1: 0.845 Max: Acc: 0.891
Procsessing: (10, 80, 10) Acc: 0.864, Precision: 0.879, Recall: 0.887, F1: 0.883 Max: Acc: 0.891
Procsessing: (10, 80, 20) Acc: 0.804, Precision: 0.776, Recall: 0.874, F1: 0.822 Max: Acc: 0.891
Procsessing: (10, 80, 40) Acc: 0.783, Precision: 0.804, Recall: 0.819, F1: 0.811 Max: Acc: 0.891
Procsessing: (10, 80, 80) Acc: 0.821, Precision: 0.776

/home/rolando/anaconda3/envs/healt/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Acc: 0.793, Precision: 0.766, Recall: 0.863, F1: 0.812 Max: Acc: 0.891
Procsessing: (10, 160, 40, 10) Acc: 0.799, Precision: 0.748, Recall: 0.889, F1: 0.812 Max: Acc: 0.891
Procsessing: (10, 160, 40, 20) Acc: 0.799, Precision: 0.757, Recall: 0.880, F1: 0.814 Max: Acc: 0.891
Procsessing: (10, 160, 40, 40) Acc: 0.783, Precision: 0.729, Recall: 0.876, F1: 0.796 Max: Acc: 0.891
Procsessing: (10, 160, 40, 80) Acc: 0.815, Precision: 0.794, Recall: 0.876, F1: 0.833 Max: Acc: 0.891
Procsessing: (10, 160, 40, 160) Acc: 0.783, Precision: 0.729, Recall: 0.876, F1: 0.796 Max: Acc: 0.891
Procsessing: (10, 160, 80, 10) Acc: 0.788, Precision: 0.729, Recall: 0.886, F1: 0.800 Max: Acc: 0.891
Procsessing: (10, 160, 80, 20) Acc: 0.804, Precision: 0.766, Recall: 0.882, F1: 0.820 Max: Acc: 0.891
Procsessing: (10, 160, 80, 40) Acc: 0.804, Precision: 0.757, Recall: 0.890, F1: 0.818 Max: Acc: 0.891
Procsessing: (10, 160, 80, 80) Acc: 0.788, Precision: 0.766, Recall: 0.854, F1: 0.808 Max: Acc: 0.891
Procsessin

In [12]:
with open('prod_4L_20-160.csv', 'w') as csvfile:
   keys = res.keys()
   for key in keys:
      csvfile.write(str(key) + ',' + str(res[key]) + '\n')

In [11]:
# lr = 0.002         # learning rate
# nn = [0, 100 ,1]  # número de neuronas por capa.

# # Creamos el objeto del modelo de red neuronal multicapa.
# clf = sknn.MLPRegressor(
#         solver='sgd', 
#         batch_size='auto',
#         tol=0.0005,
#         learning_rate_init=lr, 
#         hidden_layer_sizes=tuple(nn[1:]),
#         verbose=True,
#         n_iter_no_change=100,
#         max_iter=1000)


# # Y lo entrenamos con nuestro datos.
# clf.fit(inputs_train, outputs_train)

# acc = 0
# for i in range(0, len(inputs_test)):
#     pre = round(clf.predict(inputs_test[i].reshape(1, -1))[0], 0)
#     success = "#" if pre == outputs_test[i] else "."
#     acc = acc + 1 if pre == outputs_test[i] else acc
#     print(success, end="")

# acc = acc / len(inputs_test)
# print("\nAccuracy:", acc)